## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [153]:
import pandas as pd
import numpy as np
import sqlite3
import json
import pickle

# Initial data investigation

In [5]:
cd zippedData

[Errno 2] No such file or directory: 'zippedData'
/Users/jessicamiles/Documents/Flatiron_Data_Science/git_labs_lessons/phase1/dsc-phase-1-project/zippedData


In [6]:
ls

bom.movie_gross.csv.gz        imdb.title.principals.csv.gz
column_summary.csv            imdb.title.ratings.csv.gz
imdb.name.basics.csv.gz       rt.movie_info.tsv.gz
imdb.title.akas.csv.gz        rt.reviews.tsv.gz
imdb.title.basics.csv.gz      tmdb.movies.csv.gz
imdb.title.crew.csv.gz        tn.movie_budgets.csv.gz


In [7]:
def preview_csvtsv(filepath, row_count):
    """Create a dataframe of initial n rows from file as a preview.
    Supports either csv or tsv file formats, and files compressed as gz.
    File format and gz detected using file extensions.
    Prints a preview dataframe of the file, the preview's head(), 
    and the total number of rows.
    """
    import gzip
    is_gz = False
    
    print(f"""--------------------------------------------------
    File: {filepath}
    --------------------------------------------------""")
    
    # determine file extension(s)
    if filepath.split('.')[-1].lower() == 'csv':
        filetype='csv'
    elif filepath.split('.')[-1].lower() == 'tsv':
        filetype='tsv'
    elif filepath.split('.')[-1].lower() == 'gz':
        # if we have a gz file, get second to last element
        is_gz = True
        if filepath.split('.')[-2].lower() == 'csv':
            filetype='csv'
        elif filepath.split('.')[-2].lower() == 'tsv':
            filetype='tsv'
    else:
        # default to CSV if can't determine the file extention
        filetype='csv'
    
    # Process file based on extension. 
    # Pandas doesn't care if the file is a gz compressed
    if filetype == 'csv':
        df_test = pd.read_csv(filepath, nrows=row_count)
    elif filetype == 'tsv':
        df_test = pd.read_table(filepath, nrows=row_count)

    num_lines = 0
    
    # get line count. This differs based on if the file is gz or not
    if is_gz:
        for line in gzip.open(filepath):
            num_lines += 1
    else:
        for line in open(filepath):
            num_lines += 1
    
    display(df_test.head())
    display(df_test.info())
    print(f"\nFile contains {num_lines} total lines, including header.")
    print('\n\n\n')
    
    pass

In [8]:
def filelist_to_dfs_prelim(filepath_list, dfname_list):
    """Creates a dataframe for each file in the list.
    Names each dataframe using the corresponding name list.
    """
    df_dict = {}
    filetype = ''

    if len(filepath_list) != len(dfname_list):
        return "Error: lists must be of equivalent length."
    
    # loop through each file in the filepath_list
    for i, file in enumerate(filepath_list):
        
        print(f"Working on {file}...")
        
        # figure out the file type to use the appropriate method
        if file.split('.')[-1].lower() == 'csv':
            filetype='csv'
        elif file.split('.')[-1].lower() == 'tsv':
            filetype='tsv'
        elif file.split('.')[-2].lower() == 'csv':
            filetype='csv'
        elif file.split('.')[-2].lower() == 'tsv':
            filetype='tsv'
        else:
            # default to CSV if can't determine the file extention
            filetype='csv'
        
        try:
            # determine file extension and apply appropriate method to load preview
            if filetype == 'tsv':
                df_dict[dfname_list[i]] = pd.read_table(file)
            else:
                # default to CSV if not tsv
                df_dict[dfname_list[i]] = pd.read_csv(file) 
        except:
            print(f"Error encountered and {file} was skipped")

    return df_dict
    

In [9]:
def filelist_to_dfs_errorh(filepath_list, dfname_list):
    """Creates a dataframe for each file in the list.
    Names each dataframe using the corresponding name list.
    """
    df_dict = {}
    filetype = ''
    
    if len(filepath_list) != len(dfname_list):
        return "Error: lists must be of equivalent length."
    
    # loop through each file in the filepath_list
    for i, file in enumerate(filepath_list):
        
        print(f"Working on {file}...")

        # figure out the file type to use the appropriate method
        if file.split('.')[-1].lower() == 'csv':
            filetype='csv'
        elif file.split('.')[-1].lower() == 'tsv':
            filetype='tsv'
        elif file.split('.')[-2].lower() == 'csv':
            filetype='csv'
        elif file.split('.')[-2].lower() == 'tsv':
            filetype='tsv'
        else:
            # default to CSV if can't determine the file extention
            filetype='csv'
        
        # determine file extension and apply appropriate method to load df
        # Credit to https://stackoverflow.com/questions/33440805
        #     /pandas-dataframe-read-csv-on-bad-data for handling error rows
        if filetype == 'tsv':
            df_dict[dfname_list[i]] = pd.read_table(file
                                        , error_bad_lines = False
                                        , warn_bad_lines = True
                                        , encoding='latin1') 
        else:
            # default to CSV if not tsv
            df_dict[dfname_list[i]] = pd.read_csv(file
                                        , error_bad_lines = False
                                        , warn_bad_lines = True
                                        , encoding='latin1') 

    return df_dict
    

In [10]:
def summarize_cols(df_dict):
    """Creates a dataframe with df/column summary info.
    Takes a dictionary as input, where the key is the name of the df,
    and the value is the dataframe itself."""
    
    arr_colsumm = np.array([['df_namekey', 'col_name'
                                 , 'col_type', 'col_nan_count'
                                 , 'total_rows']])
    
    for df_key in df_dict.keys():
        # append each column's info to the array
        for col in df_dict[df_key].columns:
            arr_colsumm = np.append(arr_colsumm
               , [[df_key
                 , col
                 , df_dict[df_key][col].dtype
                 , df_dict[df_key][col].isna().sum()
                 , len(df_dict[df_key])]], axis=0)

    df_colsumm = pd.DataFrame(arr_colsumm[1:], columns=arr_colsumm[0])
    
    return df_colsumm

In [11]:
files = ['bom.movie_gross.csv.gz'
         ,'imdb.title.ratings.csv.gz'
         ,'imdb.name.basics.csv.gz'
         ,'rt.movie_info.tsv.gz'
         ,'imdb.title.akas.csv.gz'
         ,'rt.reviews.tsv.gz'
         ,'imdb.title.basics.csv.gz'
         ,'tmdb.movies.csv.gz'
         ,'imdb.title.crew.csv.gz'
         ,'tn.movie_budgets.csv.gz'
         ,'imdb.title.principals.csv.gz']

df_names = ['df_bom_gross'
         ,'df_imdb_ratings'
         ,'df_imdb_namebasics'
         ,'df_rt_movie_info'
         ,'df_imdb_akas'
         ,'df_rt_reviews'
         ,'df_imdb_basics'
         ,'df_tmdb_movies'
         ,'df_imdb_crew'
         ,'df_tn_budgets'
         ,'df_imdb_principals']

### Initial preview of first 10 rows of each file

First, I loaded only the first 10 rows of each file into a dataframe and displayed the `head()`. This function also counts the total lines in each file.

I reviewed the results from each file to understand what information they contain. In terms of what I'm hoping for, I'm not sure whether I have sufficient data to calculate the actual profit. Ideally, I would like the profit calculation to be something like:

sum(box office gross, streaming revenue, VOD revenue, merchandising revenue)

minus


sum(production actual costs, advertising spend, theaters' cut)

In [12]:
#[preview_csvtsv(file, 10) for file in files]

### Import files into data frames and summarize columns

Next, I ran a function that attempts to actually import the full files into data frames. It returns a dictionary with the keys being the df names I specified, and the values being the dataframes themselves.

One of the 'tsv' files from Rotten Tomatoes encountered errors while importing, although there were no errors with the initial 10-row preview. I was able to run through another function that has more error handling, and adjust the encoding to get a good import.

Finally, I ran the dictionary of dataframes through a function which summarizes the columns. I think this will help me see which columns I have.

In [13]:
df_dict = filelist_to_dfs_prelim(files, df_names)
df_dict.keys()

Working on bom.movie_gross.csv.gz...
Working on imdb.title.ratings.csv.gz...
Working on imdb.name.basics.csv.gz...
Working on rt.movie_info.tsv.gz...
Working on imdb.title.akas.csv.gz...
Working on rt.reviews.tsv.gz...
Error encountered and rt.reviews.tsv.gz was skipped
Working on imdb.title.basics.csv.gz...
Working on tmdb.movies.csv.gz...
Working on imdb.title.crew.csv.gz...
Working on tn.movie_budgets.csv.gz...
Working on imdb.title.principals.csv.gz...


dict_keys(['df_bom_gross', 'df_imdb_ratings', 'df_imdb_namebasics', 'df_rt_movie_info', 'df_imdb_akas', 'df_imdb_basics', 'df_tmdb_movies', 'df_imdb_crew', 'df_tn_budgets', 'df_imdb_principals'])

In [14]:
df_dict2 = filelist_to_dfs_errorh(['rt.reviews.tsv.gz'], ['df_rt_reviews'])
df_dict2.keys()

Working on rt.reviews.tsv.gz...


dict_keys(['df_rt_reviews'])

In [15]:
df_dict2['df_rt_reviews']

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"
...,...,...,...,...,...,...,...,...
54427,2000,The real charm of this trifle is the deadpan c...,NaN,fresh,Laura Sinagra,1,Village Voice,"September 24, 2002"
54428,2000,NaN,1/5,rotten,Michael Szymanski,0,Zap2it.com,"September 21, 2005"
54429,2000,NaN,2/5,rotten,Emanuel Levy,0,EmanuelLevy.Com,"July 17, 2005"
54430,2000,NaN,2.5/5,rotten,Christopher Null,0,Filmcritic.com,"September 7, 2003"


In [16]:
# Merge dict of clean dfs with the dict of dfs that had errors
df_dict.update(df_dict2)
df_dict.keys()

dict_keys(['df_bom_gross', 'df_imdb_ratings', 'df_imdb_namebasics', 'df_rt_movie_info', 'df_imdb_akas', 'df_imdb_basics', 'df_tmdb_movies', 'df_imdb_crew', 'df_tn_budgets', 'df_imdb_principals', 'df_rt_reviews'])

### Create a summary of the columns in each table

Export this summary to a CSV file so I can make outside notes.

In [17]:
df_colsumm = summarize_cols(df_dict)
#df_colsumm.to_csv('column_summary.csv',index=False)

# SQL Ingestion for deeper analysis

I will create a SQLlite database and import the files I intend to use in my analysis into tables. I think this will be the best way to clean the data and save my changes outside of the Jupyter Notebook without a lot of file import/export.

I will be making notes for what I want to transform, date ranges, etc for each table and column outside of this notebook, and then will find a good way to get the information back in. This mostly saves me from scrolling up and down and makes it easier to take notes without worrying about formatting.

## Create New SQLLite database

In [23]:
cd ..

/Users/jessicamiles/Documents/Flatiron_Data_Science/git_labs_lessons/phase1/dsc-phase-1-project


In [24]:
cd analysis

/Users/jessicamiles/Documents/Flatiron_Data_Science/git_labs_lessons/phase1/dsc-phase-1-project/analysis


In [26]:
# Create new database (or conenct to it, later)
conn = sqlite3.connect('movie_analysis.db')
cur = conn.cursor()

## Wait is this data any good??

In [30]:
# Starting with the The Numbers data set, as profit is first analysis
# I want to tackle

# Get the column info
df_dict['df_tn_budgets'].head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


Trying to determine whether the budget will always be in US dollars, whether it is adjusted for inflation, et. Trying to confirm whether domestic gross and worldwide gross include more than box office?

Interstingly, when I look up Avatar on The Numbers, the figures don't match what I see here.
https://www.the-numbers.com/movie/Avatar#tab=summary

Gross numbers more or less match what I see under box office, so I think those are box office only.

But production budget is only 237 compared to $425 MM that I see in my table.

I tried their movie comparison tool which lets you compare movies and adjust for inflation, and I'm not sure where the $425 MM is coming from, It's not just inflation. With inflation adjustment Avatar's budget is still only 289 million. https://www.the-numbers.com/movies/custom-comparisons/Avatar/Titanic-(1997)?mode=actuals#tab=day_by_day_comparison

Looking deeper at source which have APIs, The Movie Database has a free API with NO rate limiting.

I spot checked Avatar and Titanic across The Numbers and the revenue and budget are roughly equivalent (The Numbers had slightly higher revenue so may be more up to date than TMDb but pretty close.

Going to try getting the data from TMDB. Bonus is that my Watchmode data has TMDB ID, so I can easily join them.

## Looking through the rest of the data sets

### TMDB Movies

I think I'm going to get much more than this includes (definitely more fields) but curious what date ranges, etc we have.

In [27]:
df_dict['df_tmdb_movies'].head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [28]:
df_dict['df_tmdb_movies'].describe()

,Unnamed: 0,id,popularity,vote_average,vote_count
count,26517.00000,26517.000000,26517.000000,26517.000000,26517.000000
mean,13258.00000,295050.153260,3.130912,5.991281,194.224837
std,7654.94288,153661.615648,4.355229,1.852946,960.961095
min,0.00000,27.000000,0.600000,0.000000,1.000000
25%,6629.00000,157851.000000,0.600000,5.000000,2.000000
50%,13258.00000,309581.000000,1.374000,6.000000,5.000000
75%,19887.00000,419542.000000,3.694000,7.000000,28.000000
max,26516.00000,608444.000000,80.773000,10.000000,22186.000000


OK so this does go down to popularity = 0.6. Mean is 3.13, hmm.

Minimum vote_count is 1 though, so maybe it's all movies with at least one vote.

In [40]:
df_dict['df_tmdb_movies']['year'] = df_dict['df_tmdb_movies']['release_date'].map(lambda date: int(date[:4]))

In [38]:
df_dict['df_tmdb_movies']['year']

0        2010
1        2010
2        2010
3        1995
4        2010
         ... 
26512    2018
26513    2018
26514    2018
26515    2018
26516    2018
Name: year, Length: 26517, dtype: object

In [41]:
df_dict['df_tmdb_movies'].describe()

,Unnamed: 0,id,popularity,vote_average,vote_count,year
count,26517.00000,26517.000000,26517.000000,26517.000000,26517.000000,26517.000000
mean,13258.00000,295050.153260,3.130912,5.991281,194.224837,2013.953162
std,7654.94288,153661.615648,4.355229,1.852946,960.961095,3.544641
min,0.00000,27.000000,0.600000,0.000000,1.000000,1930.000000
25%,6629.00000,157851.000000,0.600000,5.000000,2.000000,2012.000000
50%,13258.00000,309581.000000,1.374000,6.000000,5.000000,2014.000000
75%,19887.00000,419542.000000,3.694000,7.000000,28.000000,2016.000000
max,26516.00000,608444.000000,80.773000,10.000000,22186.000000,2020.000000


### Rotten Tomatoes Tables

Well this would have been nice to have, but there aren't titles listed and no other data set matches its ids.

In [43]:
df_dict['df_rt_movie_info'].head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [44]:
df_dict['df_rt_reviews'].head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


### IMDB Datasets - Basics

In [45]:
df_dict['df_imdb_basics'].head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [175]:
df_dict['df_imdb_basics'].describe()

,start_year,runtime_minutes
count,146144.000000,114405.000000
mean,2014.621798,86.187247
std,2.733583,166.360590
min,2010.000000,1.000000
25%,2012.000000,70.000000
50%,2015.000000,87.000000
75%,2017.000000,99.000000
max,2115.000000,51420.000000


In [46]:
# Try importing entire dataframe into SQLLite
df_dict['df_imdb_basics'].to_sql('imdb_basics', conn,index=False)

In [47]:
conn.commit()

In [53]:
sql = """SELECT * FROM imdb_basics LIMIT 5"""

df = pd.DataFrame(cur.execute(sql).fetchall(),columns=[x[0] for x in cur.description])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           5 non-null      object 
 1   primary_title    5 non-null      object 
 2   original_title   5 non-null      object 
 3   start_year       5 non-null      int64  
 4   runtime_minutes  4 non-null      float64
 5   genres           5 non-null      object 
dtypes: float64(1), int64(1), object(4)
memory usage: 368.0+ bytes


### IMDB Datasets - Crew

In [57]:
df_dict['df_imdb_crew'].head()

,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,NaN,"nm0175726,nm1802864"
2,tt0462036,nm1940585,nm1940585
3,tt0835418,nm0151540,"nm0310087,nm0841532"
4,tt0878654,"nm0089502,nm2291498,nm2292011",nm0284943


In [61]:
print(len(df_dict['df_imdb_crew']['tconst'].unique()))
print(len(df_dict['df_imdb_crew']))

146144
146144


The imdb_crew table matches up `tconst` (movie IDs) with `nconst` (name IDs). However, although each movie is listed once (`tconst` is unique, the people are comma-separated lists. Need to split this into separate lookup tables where they're flattened, so I can join one-to-many and select individual writers and directors independent of specific movies.

In [62]:
# Make a new working dataframe
df_crew = df_dict['df_imdb_crew']

In [72]:
# Create a new column with the directors converted to a list
df_crew['directors_list'] = df_crew[
    df_crew['directors'].isna() == False]['directors'].map(
    lambda x: list(x.split(',')))
df_crew.head()

,tconst,directors,writers,directors_list
0,tt0285252,nm0899854,nm0899854,[nm0899854]
1,tt0438973,NaN,"nm0175726,nm1802864",NaN
2,tt0462036,nm1940585,nm1940585,[nm1940585]
3,tt0835418,nm0151540,"nm0310087,nm0841532",[nm0151540]
4,tt0878654,"nm0089502,nm2291498,nm2292011",nm0284943,"[nm0089502, nm2291498, nm2292011]"


In [73]:
# Create a new column with the writers converted to a list
df_crew['writers_list'] = df_crew[
    df_crew['writers'].isna() == False]['writers'].map(
    lambda x: list(x.split(',')))
df_crew.head()

,tconst,directors,writers,directors_list,writers_list
0,tt0285252,nm0899854,nm0899854,[nm0899854],[nm0899854]
1,tt0438973,NaN,"nm0175726,nm1802864",NaN,"[nm0175726, nm1802864]"
2,tt0462036,nm1940585,nm1940585,[nm1940585],[nm1940585]
3,tt0835418,nm0151540,"nm0310087,nm0841532",[nm0151540],"[nm0310087, nm0841532]"
4,tt0878654,"nm0089502,nm2291498,nm2292011",nm0284943,"[nm0089502, nm2291498, nm2292011]",[nm0284943]


In [77]:
# Create yet another dataframe with just the directors and remove NaNs
df_directors = df_crew.loc[df_crew['directors_list'].isna() == False, ['tconst'
                                                        ,'directors_list']]
df_directors.isna().any().sum()

0

In [81]:
# Use explode to create a row for each director, with tconst duplicated
df_directors = df_directors.explode('directors_list', ignore_index=True)

print(f"Unique: {df_directors['tconst'].nunique()}")
print(f"Total Rows: {len(df_directors)}")
df_directors.head()

Unique: 140417
Total Rows: 163535


,tconst,directors_list
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0878654,nm0089502
4,tt0878654,nm2291498


In [95]:
df_directors.rename(columns={'directors_list':'nconst'}, inplace=True)

In [96]:
# Import our squeaky clean dataframe into SQL

df_directors.to_sql('imdb_directors', conn ,index=False)

sql = """SELECT * FROM imdb_directors LIMIT 5"""

df = pd.DataFrame(cur.execute(sql).fetchall(),columns=[x[0] for x in cur.description])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tconst  5 non-null      object
 1   nconst  5 non-null      object
dtypes: object(2)
memory usage: 208.0+ bytes


In [97]:
conn.commit()

In [89]:
# Same deal with writers

# Create yet another dataframe with just the writers and remove NaNs
df_writers = df_crew.loc[df_crew['writers_list'].isna() == False, ['tconst'
                                                        ,'writers_list']]

# Use explode to create a row for each writers, with tconst duplicated
df_writers = df_writers.explode('writers_list', ignore_index=True)

print(f"Unique: {df_writers['tconst'].nunique()}")
print(f"Total Rows: {len(df_writers)}")

Unique: 110261
Total Rows: 178352


In [98]:
df_writers.rename(columns={'writers_list':'nconst'},inplace=True)

In [99]:
df_writers.to_sql('imdb_writers', conn ,index=False)

sql = """SELECT * FROM imdb_writers LIMIT 5"""

df = pd.DataFrame(cur.execute(sql).fetchall(),columns=[x[0] for x in cur.description])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tconst  5 non-null      object
 1   nconst  5 non-null      object
dtypes: object(2)
memory usage: 208.0+ bytes


In [100]:
conn.commit()

### IMDB Datasets - Name Basics

In [101]:
df_dict['df_imdb_namebasics'].head()

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"


In [104]:
df_name = df_dict['df_imdb_namebasics'][['nconst', 'primary_name', 'birth_year']]
df_name.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   nconst        606648 non-null  object 
 1   primary_name  606648 non-null  object 
 2   birth_year    82736 non-null   float64
dtypes: float64(1), object(2)
memory usage: 13.9+ MB


In [ ]:
column_types = {'nconst':'TEXT', 'primary_name': 'TEXT', 'birth_year': 'INTEGER'}
df_name.to_sql('imdb_names', conn ,index=False, dtype=column_types)

In [111]:
sql = """SELECT * 
         FROM imdb_names 
         WHERE birth_year IS NOT NULL
         LIMIT 5"""

df = pd.DataFrame(cur.execute(sql).fetchall(),columns=[x[0] for x in cur.description])
df

,nconst,primary_name,birth_year
0,nm0063618,Jeff Beal,1963
1,nm0065847,Michael Frost Beckner,1963
2,nm0066093,Ahmad Bedair,1945
3,nm0066897,Mohammad-Ali Behboudi,1956
4,nm0067983,Krunoslav Belko,1971


In [114]:
conn.commit()

### IMDB Datasets - Principals

In [112]:
df_dict['df_imdb_principals'].head()

,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN,NaN
2,tt0111414,3,nm3739909,producer,producer,NaN
3,tt0323808,10,nm0059247,editor,NaN,NaN
4,tt0323808,1,nm3579312,actress,NaN,"[""Beth Boothby""]"


In [132]:
print(len(df_dict['df_imdb_principals']['nconst'].unique()))
print(len(df_dict['df_imdb_principals']))

604546
1028151


Hmm, so `tconst` definitely isn't unique, and `nconst` isn't unique either. I'm thinking maybe this is already exploded, and the duplicates are cases where a given person was both an actor and director, or held multiple roles. Let's check.

In [130]:
df_ppals = df_dict['df_imdb_principals']

In [131]:
# Let's preview some of the rows where nconst and tconst and category are the same

df_ppals[df_ppals.duplicated(subset=['nconst', 'tconst','category'], keep=False) == True]

,tconst,ordering,nconst,category,job,characters


In [125]:
# I don't want to import duplicate data, so will drop duplicates for the 
# columns I will be using

df_ppals.drop_duplicates(subset=['nconst', 'tconst','category']
                         , keep='first', inplace=True)

In [133]:
df_ppals[['tconst','nconst', 'category']]

,tconst,nconst,category
0,tt0111414,nm0246005,actor
1,tt0111414,nm0398271,director
2,tt0111414,nm3739909,producer
3,tt0323808,nm0059247,editor
4,tt0323808,nm3579312,actress
...,...,...,...
1028181,tt9692684,nm0186469,actor
1028182,tt9692684,nm4929530,self
1028183,tt9692684,nm10441594,director
1028184,tt9692684,nm6009913,writer


In [138]:
df_ppals['category'].value_counts()

"""
actor                  256700
director               146393
actress                146193
producer               113723
cinematographer         80091
composer                77063
writer                  74357
self                    65424
editor                  55511
production_designer      9373
archive_footage          3307
archive_sound              16
"""

'actor                  256700\ndirector               146393\nactress                146193\nproducer               113723\ncinematographer         80091\ncomposer                77063\nwriter                  74357\nself                    65424\neditor                  55511\nproduction_designer      9373\narchive_footage          3307\narchive_sound              16\nName: category, dtype: int64'

In [141]:
df_ppals[['tconst','nconst', 'category']].to_sql('imdb_principals', conn ,index=False)

sql = """SELECT * 
         FROM imdb_principals
         LIMIT 5"""

df = pd.DataFrame(cur.execute(sql).fetchall(),columns=[x[0] for x in cur.description])
df

,tconst,nconst,category
0,tt0111414,nm0246005,actor
1,tt0111414,nm0398271,director
2,tt0111414,nm3739909,producer
3,tt0323808,nm0059247,editor
4,tt0323808,nm3579312,actress


In [142]:
conn.commit()

### IMDB Datasets - Ratings

In [145]:
df_dict['df_imdb_ratings'].head()

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [152]:
df_dict['df_imdb_ratings'].describe()

,averagerating,numvotes
count,73856.000000,7.385600e+04
mean,6.332729,3.523662e+03
std,1.474978,3.029402e+04
min,1.000000,5.000000e+00
25%,5.500000,1.400000e+01
50%,6.500000,4.900000e+01
75%,7.400000,2.820000e+02
max,10.000000,1.841066e+06


In [149]:
df_dict['df_imdb_ratings'].to_sql('imdb_ratings', conn ,index=False)

sql = """SELECT * 
         FROM imdb_ratings
         LIMIT 5"""

df = pd.DataFrame(cur.execute(sql).fetchall(),columns=[x[0] for x in cur.description])
df

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [150]:
conn.commit()

## Importing API-retrieved TMDB data

In [222]:
def df_str_to_iterable(df, trans_col, new_col, function):
    temp_s = df[df[trans_col].isna() == False][trans_col].map(function)
    df.insert(loc=len(df.columns), column=new_col, value=temp_s)
    return None

In [156]:
cd data

/Users/jessicamiles/Documents/Flatiron_Data_Science/git_labs_lessons/phase1/dsc-phase-1-project/data


In [157]:
batch1 = pickle.Unpickler(open('tmdb_moviedetails_batch1.pickle', 'rb')).load()

df_batch1 = pd.DataFrame(batch1)
df_batch1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10106 entries, 0 to 10105
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   adult                 10106 non-null  bool   
 1   id                    10106 non-null  int64  
 2   original_title        10106 non-null  object 
 3   popularity            10106 non-null  float64
 4   video                 10106 non-null  bool   
 5   budget                10106 non-null  int64  
 6   genres                10106 non-null  object 
 7   homepage              5 non-null      object 
 8   imdb_id               10106 non-null  object 
 9   original_language     10106 non-null  object 
 10  overview              10106 non-null  object 
 11  production_companies  10106 non-null  object 
 12  production_countries  10106 non-null  object 
 13  release_date          10106 non-null  object 
 14  revenue               10106 non-null  int64  
 15  runtime            

In [170]:
df_batch1.head()

,id,original_title,popularity,video,budget,genres,imdb_id,original_language,overview,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count
0,2,Ariel,8.455,False,0,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",tt0094675,fi,Taisto Kasurinen is a Finnish coal miner whose...,"[{'id': 2303, 'logo_path': None, 'name': 'Vill...","[{'iso_3166_1': 'FI', 'name': 'Finland'}]",1988-10-21,0,73.0,Released,,Ariel,6.8,128
1,3,Varjoja paratiisissa,6.696,False,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",tt0092149,fi,"An episode in the life of Nikander, a garbage ...","[{'id': 2303, 'logo_path': None, 'name': 'Vill...","[{'iso_3166_1': 'FI', 'name': 'Finland'}]",1986-10-17,0,74.0,Released,,Shadows in Paradise,7.2,125
2,5,Four Rooms,13.794,False,4000000,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",tt0113101,en,It's Ted the Bellhop's first night on the job....,"[{'id': 14, 'logo_path': '/m6AHu84oZQxvq7n1rsv...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-09,4257354,98.0,Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,5.7,1901
3,6,Judgment Night,12.455,False,21000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",tt0107286,en,"While racing to a boxing match, Frank, Mike, J...","[{'id': 33, 'logo_path': '/8lvHyhjr8oUKOOy2dKX...","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",1993-10-15,12136938,110.0,Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,6.5,184
4,11,Star Wars,84.275,False,11000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",tt0076759,en,Princess Leia is captured and held hostage by ...,"[{'id': 1, 'logo_path': '/o86DbpburjxrqAzEDhXZ...","[{'iso_3166_1': 'US', 'name': 'United States o...",1977-05-25,775398007,121.0,Released,"A long time ago in a galaxy far, far away...",Star Wars,8.2,15257


In [ ]:
df_batch1[df_batch1['adult']==True]

In [169]:
df_batch1.drop(columns=['adult', 'homepage'], inplace=True)

In [188]:
# Need to parse the genres, production_companies, and production_countries columns
# these are all dictionaries and I probably only want parts of them into lists
# , which I can then explode
# Also need to convert the release_date to a date

type(df_batch1.at[2, 'genres'])

list

In [219]:
# initial view of genres column. Separated out for working
df_batch1g = df_batch1[['id', 'genres']]
df_batch1g.head()

,id,genres
0,2,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name..."
1,3,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name..."
2,5,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name..."
3,6,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam..."
4,11,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '..."


In [223]:
# Add new column with genre IDs as a list
myfunc = lambda x: [i['id'] for i in x]
df_str_to_iterable(df_batch1g, 'genres', 'genreids_list', myfunc)

In [235]:
# Create a dictionary and start collecting all the ids and names for the genres
# eventually will write this to the table
#genres_dict = {}

In [237]:
for value in df_batch1g['genres'].values:
    for x in value:
        genres_dict[x['id']] = x['name']

genres_dict.values()

dict_values(['Drama', 'Crime', 'Comedy', 'Action', 'Thriller', 'Adventure', 'Science Fiction', 'Animation', 'Family', 'Romance', 'Mystery', 'Horror', 'Fantasy', 'Documentary', 'War', 'Music', 'History', 'Western', 'TV Movie'])

In [240]:
df_batch1g_ex = df_batch1g[['id', 'genreids_list']].explode('genreids_list'
                                                           , ignore_index=True)

df_batch1g_ex

,id,genreids_list
0,2,18
1,2,80
2,2,35
3,3,18
4,3,35
...,...,...
24570,20859,53
24571,20860,35
24572,20861,18
24573,20862,18


In [178]:
df_batch1.at[2, 'production_companies']

[{'id': 14,
  'logo_path': '/m6AHu84oZQxvq7n1rsvMNJIAsMu.png',
  'name': 'Miramax',
  'origin_country': 'US'},
 {'id': 59,
  'logo_path': '/yH7OMeSxhfP0AVM6iT0rsF3F4ZC.png',
  'name': 'A Band Apart',
  'origin_country': 'US'}]

In [244]:
# initial view of prodcomp column. Separated out for working
df_batch1pcomp = df_batch1[['id', 'production_companies']]
df_batch1pcomp.head()

# Add new column with prodcomp IDs as a list
myfunc = lambda x: [i['id'] for i in x]
df_str_to_iterable(df_batch1pcomp, 'production_companies', 'prodcomp_list', myfunc)

# Create a dictionary and start collecting all the ids and names for the prodcoms
# eventually will write this to the table
prodcomps_dict = {}

for value in df_batch1pcomp['production_companies'].values:
    for x in value:
        prodcomps_dict[x['id']] = [x['name'], x['origin_country']]

df_batch1pcomp_ex = df_batch1pcomp[['id', 'prodcomp_list']].explode('prodcomp_list'
                                                           , ignore_index=True)

df_batch1pcomp_ex

,id,prodcomp_list
0,2,2303
1,3,2303
2,5,14
3,5,59
4,6,33
...,...,...
24447,20859,16953
24448,20860,NaN
24449,20861,NaN
24450,20862,12269


In [180]:
df_batch1.at[3, 'production_countries']

[{'iso_3166_1': 'JP', 'name': 'Japan'},
 {'iso_3166_1': 'US', 'name': 'United States of America'}]

In [246]:
# initial view of prodcoun column. Separated out for working
df_batch1pcoun = df_batch1[['id', 'production_countries']]
df_batch1pcoun.head()

# Add new column with prodcoun IDs as a list
myfunc = lambda x: [i['iso_3166_1'] for i in x]
df_str_to_iterable(df_batch1pcoun, 'production_countries', 'prodcouns_list', myfunc)

# Create a dictionary and start collecting all the ids and names for the prodcouns
# eventually will write this to the table
prodcouns_dict = {}

for value in df_batch1pcoun['production_countries'].values:
    for x in value:
        prodcouns_dict[x['iso_3166_1']] = x['name']

df_batch1pcoun_ex = df_batch1pcoun[['id', 'prodcouns_list']].explode('prodcouns_list'
                                                           , ignore_index=True)

df_batch1pcoun_ex

,id,prodcouns_list
0,2,FI
1,3,FI
2,5,US
3,6,JP
4,6,US
...,...,...
13563,20859,CA
13564,20860,NaN
13565,20861,NaN
13566,20862,CA


In [248]:
df_batch1.head(1)

,id,original_title,popularity,video,budget,genres,imdb_id,original_language,overview,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count
0,2,Ariel,8.455,False,0,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",tt0094675,fi,Taisto Kasurinen is a Finnish coal miner whose...,"[{'id': 2303, 'logo_path': None, 'name': 'Vill...","[{'iso_3166_1': 'FI', 'name': 'Finland'}]",1988-10-21,0,73.0,Released,,Ariel,6.8,128


In [251]:
cols_list = ['id', 'original_title', 'popularity', 'video', 'budget',
        'imdb_id', 'original_language', 'overview','release_date', 'revenue', 
        'runtime', 'status', 'tagline', 'title', 'vote_average', 'vote_count']
df_batch1[cols_list].to_sql('tmdb_movies', conn ,index=False)

sql = """SELECT * 
         FROM tmdb_movies
         LIMIT 5"""

df = pd.DataFrame(cur.execute(sql).fetchall(),columns=[x[0] for x in cur.description])
df

,id,original_title,popularity,video,budget,imdb_id,original_language,overview,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count
0,2,Ariel,8.455,0,0,tt0094675,fi,Taisto Kasurinen is a Finnish coal miner whose...,1988-10-21,0,73.0,Released,,Ariel,6.8,128
1,3,Varjoja paratiisissa,6.696,0,0,tt0092149,fi,"An episode in the life of Nikander, a garbage ...",1986-10-17,0,74.0,Released,,Shadows in Paradise,7.2,125
2,5,Four Rooms,13.794,0,4000000,tt0113101,en,It's Ted the Bellhop's first night on the job....,1995-12-09,4257354,98.0,Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,5.7,1901
3,6,Judgment Night,12.455,0,21000000,tt0107286,en,"While racing to a boxing match, Frank, Mike, J...",1993-10-15,12136938,110.0,Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,6.5,184
4,11,Star Wars,84.275,0,11000000,tt0076759,en,Princess Leia is captured and held hostage by ...,1977-05-25,775398007,121.0,Released,"A long time ago in a galaxy far, far away...",Star Wars,8.2,15257


In [252]:
df_batch1g_ex.to_sql('tmdb_movie_genres', conn ,index=False)

sql = """SELECT * 
         FROM tmdb_movie_genres
         LIMIT 5"""

df = pd.DataFrame(cur.execute(sql).fetchall(),columns=[x[0] for x in cur.description])
df

,id,genreids_list
0,2,18
1,2,80
2,2,35
3,3,18
4,3,35


In [255]:
df_batch1pcomp_ex.to_sql('tmdb_movie_companies', conn ,index=False)

sql = """SELECT * 
         FROM tmdb_movie_companies
         LIMIT 5"""

df = pd.DataFrame(cur.execute(sql).fetchall(),columns=[x[0] for x in cur.description])
df

,id,prodcomp_list
0,2,2303
1,3,2303
2,5,14
3,5,59
4,6,33


In [256]:
df_batch1pcoun_ex.to_sql('tmdb_movie_countries', conn ,index=False)

sql = """SELECT * 
         FROM tmdb_movie_countries
         LIMIT 5"""

df = pd.DataFrame(cur.execute(sql).fetchall(),columns=[x[0] for x in cur.description])
df

,id,prodcouns_list
0,2,FI
1,3,FI
2,5,US
3,6,JP
4,6,US


In [257]:
conn.commit()

#### One-off Fix for batch 8 with the error

In [299]:
batch = pickle.Unpickler(open('tmdb_moviedetails_batch8.pickle', 'rb')).load()

df_batch = pd.DataFrame(batch)

In [302]:
df_batch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10130 entries, 0 to 10129
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   adult                 10130 non-null  bool   
 1   id                    10130 non-null  int64  
 2   original_title        10130 non-null  object 
 3   popularity            10130 non-null  float64
 4   video                 10130 non-null  bool   
 5   budget                10130 non-null  object 
 6   genres                10130 non-null  object 
 7   imdb_id               8690 non-null   object 
 8   original_language     10130 non-null  object 
 9   overview              10130 non-null  object 
 10  production_companies  10130 non-null  object 
 11  production_countries  10130 non-null  object 
 12  release_date          10130 non-null  object 
 13  revenue               10130 non-null  object 
 14  runtime               10006 non-null  object 
 15  status             

In [303]:
df_batch.loc[df_batch['id'] == 644910]

,adult,id,original_title,popularity,video,budget,genres,imdb_id,original_language,overview,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count
5432,False,644910,Saturday Morning Cartoons: 1970s — Volume 1,6.65,False,Exception encountered,Exception encountered,Exception encountered,Exception encountered,Exception encountered,Exception encountered,Exception encountered,Exception encountered,Exception encountered,Exception encountered,Exception encountered,Exception encountered,Exception encountered,Exception encountered,Exception encountered


In [305]:
df_batch.drop(index=5432, axis=0, inplace=True)

In [309]:
fixed_batch8 = df_batch.to_dict(orient='records')

pickle.Pickler(open(f'tmdb_moviedetails_batch8_fixed.pickle', 'wb')).dump(fixed_batch8)

### The Batch Import and Clean

In [310]:
# all together now!
batch = pickle.Unpickler(open('tmdb_moviedetails_batch8_fixed.pickle', 'rb')).load()

df_batch = pd.DataFrame(batch)

df_batchg = df_batch[['id', 'genres']]

# Add new column with genre IDs as a list
myfunc = lambda x: [i['id'] for i in x]
df_str_to_iterable(df_batchg, 'genres', 'genreids_list', myfunc)

for value in df_batchg['genres'].values:
    for x in value:
        genres_dict[x['id']] = x['name']

df_batchg_ex = df_batchg[['id', 'genreids_list']].explode('genreids_list'
                                                           , ignore_index=True)

df_batchpcomp = df_batch[['id', 'production_companies']]

# Add new column with prodcomp IDs as a list
myfunc = lambda x: [i['id'] for i in x]
df_str_to_iterable(df_batchpcomp, 'production_companies', 'prodcomp_list', myfunc)

for value in df_batchpcomp['production_companies'].values:
    for x in value:
        prodcomps_dict[x['id']] = [x['name'], x['origin_country']]

df_batchpcomp_ex = df_batchpcomp[['id', 'prodcomp_list']].explode('prodcomp_list'
                                                           , ignore_index=True)

df_batchpcoun = df_batch[['id', 'production_countries']]

# Add new column with prodcoun IDs as a list
myfunc = lambda x: [i['iso_3166_1'] for i in x]
df_str_to_iterable(df_batchpcoun, 'production_countries', 'prodcouns_list', myfunc)

for value in df_batchpcoun['production_countries'].values:
    for x in value:
        prodcouns_dict[x['iso_3166_1']] = x['name']

df_batchpcoun_ex = df_batchpcoun[['id', 'prodcouns_list']].explode('prodcouns_list'
                                                           , ignore_index=True)


In [314]:
print(df_batchg_ex.head())
print(df_batchpcomp_ex.head())
print(df_batchpcoun_ex.head())

       id genreids_list
0  531208            28
1  531208            27
2  531208           878
3  531210            16
4  531219         10751
       id prodcomp_list
0  531208          1311
1  531210           NaN
2  531219         11395
3  531219           174
4  531219        101082
       id prodcouns_list
0  531208             US
1  531210            NaN
2  531219             MX
3  531219             US
4  531239             US


In [315]:
df_batchg_ex.to_sql('tmdb_movie_genres', conn ,index=False, if_exists='append')

df_batchpcomp_ex.to_sql('tmdb_movie_companies', conn ,index=False, if_exists='append')

df_batchpcoun_ex.to_sql('tmdb_movie_countries', conn ,index=False, if_exists='append')

In [317]:
sql = """SELECT COUNT() 
         FROM tmdb_movies"""

df = pd.DataFrame(cur.execute(sql).fetchall(),columns=[x[0] for x in cur.description])
df

,COUNT()
0,10106


In [319]:
conn.commit()

In [148]:
#sql = """DROP TABLE imdb_ratings"""

cur.execute(sql)

In [258]:
sql = """SELECT name FROM sqlite_master WHERE type = 'table'"""

print(cur.execute(sql).fetchall())

[('imdb_basics',), ('imdb_directors',), ('imdb_writers',), ('imdb_names',), ('imdb_principals',), ('imdb_ratings',), ('tmdb_movies',), ('tmdb_movie_genres',), ('tmdb_movie_companies',), ('tmdb_movie_countries',)]


# Initial Analysis Notes and thoughts:
The question I am asked to investigate is: "which movies are doing best at the box office?" But if the business goal is successfully entering the movie-making market, this question is only one of many we should ask and answer to determine the best approach.

Considering only the past earnings from box office sales is unlikely to be a good predictor of successful movies in the future. Even if we were to use only the past few years of data, the business model is rapidly changing. With the rise of streaming, many movies aren't released in theaters at all anymore, and the future of movie theaters has become especially uncertain due to the Covid-19 pandemic. 

The attributes of movies that are the most successful in the theaters may vary from the attributes of movies that are most successful when released only on streaming platforms. The reasons why people may choose to go and watch a movie in the theaters will be different from the reasons people choose to stream a movie: theaters offer an event-like experience with the very large screen and loud surround sound, whereas viewers streaming movies watch them in their homes. 

It's unclear whether the entire business model of creating movies for streaming services differs from the traditional model. 

## Look deeper into what we have in each data set

Answer questions such as:
- What information is in each data set? Does any single data set have everything you need, or do you need to figure out how to join data sets?
- What is the range of release years in each data set? Which have the most data?
- What data points do I want to focus on in each data set, and what can I discard/disregard? This will narrow down the data I intend to clean up.

1. Monetary stuff in The Numbers data set - box office gross and budget. Look at changes over time to attempt to identify trends.
    - Is there a sweet spot in terms of a  budget range to maximize profit? Are there diminishing returns?
    - Do we see any trends in profits over time? Has the distribution of profits coming from domestic versus foreign changed over time?
2. What are the attributes of the most profitable movies? Do they get higher critical ratings? Is there a certain sweet spot for runtime length? How about MPAA ratings? Genres? Release month? <-- not sure we'll have MPAA ratings, but we'll have runtime, genres, and release date.
    - Should the studio focus on making movies that make critics happy to maximize profits? <-- not sure we'll be able to get this. but we can get public voter ratings.
3. What can we learn about the contributors to movies? If we look at the writers, directors, producers on the most profitable movies, then look at other movies they were in, do we see any patterns? How about the actors and actresses? Do more experienced contributors (those who have worked on x amount of movies prior) create a more profitable product?
    - Should Microsoft focus on hiring certain types of talent? Should they consider paying a premium for more famous actors or directors?


## Measures of successful movies include:

|Measure|Have it?|Where|Notes|
|---|---|---|---|
|Gross earnings (box office only)|Yes|bom.movie_gross.csv.gz, tn.movie_budgets.csv.gz, rt_movie_info|BOM splits into foreign and domestic. TN has the most records, AND prod budget. RT has only 1500.|
|Net box office earnings after subtracting costs|Some|Can calculate from gross - production budget|Other data sets have gross box office earnings but not production budget|df_tn_budget|
|Merchandising earnings|No|---|---|
|Gross earnings from other sources, such as streaming, rentals|No|---|
|High ratings from professional movie reviewers|Yes|rt.reviews.tsv.gz|RT is the only data set with critical reviews|
|High ratings from viewers|Yes|imdb.title.ratings.csv.gz, tmdb.movies.csv.gz|I think the tmdb ratings are users not critics|
|Being nominated for and winning awards|No|---|
|---|---|---|
 
Microsoft will need to decide which measures of success are most important. Given its primary business is profitable and in a totally different economic sector, it may decide to treat the foray into film as a way to keep the brand feeling "cool" to consumers. If that's the case, measures of success such as ratings and award nominations may be weighted more heavily than profit.

## What measures of success would a film STUDIO care about?

https://www.entrepreneur.com/article/288819 accessed march 15, 2021

"In short, the studio approach to filmmaking is a far cry from the ragtag process many independent films follow. Movie studios invest in scripts and concepts, then hire production companies to use their resources to create those films. They handle logistics like financing, production and distribution to make things easier on the creative minds behind the films."

In addition to profitability, **which production companies have been behind the most profitable films?**

The **specific writers and directors associated with profitable movies** may also be a factor studios would care about.

## Potential Independent Variables
Reasons why certain movies may be more successful than others (independent variables) include:

|Independent Variable|Have it?|Where|Notes|
|---|---|---|---|
|Budget - i.e. how much money they spent making it|Yes|tn.movie_budgets.csv.gz|
|Genre (action, drama, comedy, etc)|Yes|imdb.title.basics.csv.gz|
|Well-known and loved actors being in them|Yes, ish|imdb.name.basics.csv.gz, imdb.title.principals.csv.gz|Join tables to figure out who did what|
|Well-known and loved writers, directors, and other creators|Yes|imdb.name.basics.csv.gz,imdb.title.crew.csv.gz||
|Seasonality (combination of topic and release date)|No|---|
|Whether the movie is part of a series or franchise|Can get|---|Perhaps if based on a book or comic book? https://www.the-numbers.com/movies/franchises|
|Month of release|Yes|tmdb.movies.csv.gz|
|Method of release (in theaters only, straight to streaming, etc)|No|---|
|MPAA Rating|Yes|rt.movie_info.tsv.gz|
|Runtime length|Yes|rt.movie_info.tsv.gz, imdb.title.basics.csv.gz|
|Semantic characteristics of the title|Yes|imdb.title.akas.csv.gz|---|
|Semantic characteristics of the official synopsis|Yes|rt.movie_info.tsv.gz|---|



## Potential Trends to Consider
- Has there been any change in the popularity of movies directed by different categories of people? For instance, are movies written or directed by women becoming more popular over time?
- How is the method of release changing over time?
- Are budgets changing over time? Is the change in line with inflation, or changing at a higher rate?
- Is the general profitability changing over time? Should Microsoft rethink getting into movies if the profit margins are getting slimmer and slimmer?
- How many very successful movies are made by smaller, independent studios? How long have the most successful studios been established?
- Is there a really big gap between the highest earners / most popular and the average? Do the movies we consider successful stand way out from the pack? What's the standard deviation?

## Sources used for research
- In general, major studios don’t disclose the full budgets for their films (production, development, marketing, and advertising).[1]
- In addition to the cost of production, the gross box office earnings don't factor in that the theaters keep a cut of the ticket sales. It's hard to know what the production company actually gets.[1]
- "Traditionally, a larger chunk went to the studio during the opening weekend of a film. As the weeks went on, the theater operator's percentage rose. A studio might make about 60% of a film's ticket sales in the United States, and around 20% to 40% of that on overseas ticket sales." [1]
- "Studios and distributors generally make more from domestic revenue than from overseas sales because they get a larger percentage. Despite this arrangement, foreign ticket sales became more important in the early 21st century.6﻿ That is part of the reason why you see more sci-fi, adventure, fantasy, and superhero movies. Action and special effects require no translations. They’re easy to understand, whether you’re in Malaysia or Montana. It is much harder to build a foreign audience for an indie comedy." [1]
- The movie industry is rather famously difficult to predict accurately. Many have tried, but it's difficult to do. [2]
- One hit can pay for a lot of flops [2]
- Many of the existing production companies have been around for a long time [2]
-"Because it’s harder to get financing and audiences, companies are competing to make bigger, costlier films while eliminating risk, which is why ever-more movies are based on existing intellectual property. Eighteen of the all-time 100 top-grossing movies (adjusted for inflation) were sequels, and more than half of those were released since 2000." [2]


[1]: Source = https://www.investopedia.com/articles/investing/093015/how-exactly-do-movies-make-money.asp#:~:text=VOD%20revenues%20tend%20to%20dry,audiences%20away%20from%20traditional%20movies Accessed on 3/12/2021, 3/15/2021.

[2] Source = https://www.nytimes.com/2012/07/01/magazine/how-does-the-film-industry-actually-make-money.html Accessed 3/12/2021.

 